## LDA(Latent Dirichlet Allocation), 잠재디리클레할당
    - 토픽 모델링에 사용되는 대표적인 알고리즘
    
- 모듈
    - gensim.models.wrappers.LdaMallet 모듈사용 => 깁스 샘플링을 사용하여 훈련 코퍼스에서 LDA 모델 추천가능
        - 깁스 샘플링 => 2번째로 생성될 표본은 1차 생성 표본의 영향을 받으며, 나머지 변수는 가만히 두고 한 변수에만 변화를 준다.
            - ex) p(x1, y1, z1)가 존재할 경우, 변수명+표본의 차수
            - 변경 순서 => p(y1, z1)을 유지시킨 채 x1 -> x2 로 변경 => p(x2, z1)을 유지시킨 채 y1 -> y2 로 변경 => ...
            - 1차 변경 결과 => p(x2, y2, z2)

        - 검색창에 mallet.cs.umass.edu/dist/mallet-2.0.8.zip 치면 해당 zip파일 자동다운로드
        
        
- CoherenceMetric
	- 토픽 내 일관성 지수를 계산, 최적의 토픽 개수를 찾기위해서 사용
	- coherence = 'c_v' 사용
	- score가 높은 토픽 개수 사용
    

- Corpora 라이브러리
	- from gensim import corpora / from gensim.corpora.dictionary import Dictionary
	- 텍스트 데이터의 특성 상 수치화해서 분석 및 연산 진행해야함.
	- 텍스트를 가지고 분석이나 연산에 용이하게 데이터 집합을 만들어 주는 라이브러리
	- document = ['a', 'b', 'c']라는 리스트가 존재할 때
	- dict = corpora.Dictionary(document) / dict = Dictionary(document) 
	- dict는 리스트가 사전형식으로 변환 (ex) {'a': 0, 'b': 1, 'c': 2}
	- doc2bow를 통해서 단어를 숫자로 변경하고 카운트까지 알 수 있음
	- dict.doc2bow(input) 으로 input에 있는 텍스트 사전을 통해 단어별 개수 확인
	- ex) [(0, 1), (1, 5)]이면 0번에 위치한 단어는 1번, 1번에 위치한 단어는 5번 나왔다는 의미

In [1]:
# !pip install konlpy
# 3.8.5 버전이후로 gensim.models.wrappers.LdaMallet 삭제 => 3.8.3 버전 사용
# !pip install gensim==3.8.3

In [1]:
from tqdm.notebook import tqdm # 진행률 프로세스바
import itertools # 효율적인 루핑을 위한 이터레이터
from gensim.models.ldamodel import LdaModel # LDA 모델
from gensim.models.callbacks import CoherenceMetric 
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel # 최적의 토픽개수 구하는 모델
import pandas as pd
import numpy as np
from konlpy.tag import Okt
import os

In [5]:
# 전체 데이터 읽기
# df = pd.read_parquet('after_preprocessing.parquet')

# 필요한 컬럼만 읽고 싶을 경우
df = pd.read_parquet('after_preprocessing.parquet', columns=['source', 'content'])

In [6]:
# Okt로 토큰화
def tokenizedOkt(tourDf):
    okt = Okt()
    tour_tokenized = [] 

    # 해당 관광지에 관한 content들 토큰화
    tqdmDf = tqdm(tourDf)

    for tourInfo in tqdmDf:
        tqdmDf.set_description(f'Processing tokenized')
        tokenized_doc = okt.pos(tourInfo)
        
        # 명사만 사용
        tokenized_list = [w for w,t in tokenized_doc if t =='Noun']
        tour_tokenized.append(tokenized_list)
        
    return tour_tokenized

In [7]:
# 토크나이징한 데이터를 입력으로 받아서, 이 데이터들을 바탕으로 사전 제작 
# corpus에는 (token_id, token_count)가 튜플형태로 저장
def dictionary(data_word):
    
    # 토큰화된 문장리스트로 사전구성
    id2word=corpora.Dictionary(data_word)
    id2word.filter_extremes(no_below = 20) # 20회 이하로 등장한 단어는 삭제
    texts = data_word
    # 토큰화된 문장리스트의 각 단어를 통해서 해당 단어가 총 몇번 등장하는 지 계산
    corpus=[id2word.doc2bow(text) for text in texts]
    return id2word, texts, corpus

In [39]:
def LDA(data_word, Data_list, num_topics):
    id2word, texts, corpus = dictionary(data_word)
    
    os.environ.update({'MALLET_HOME':r'C:/pythonPrj/jejuModel/mallet-2.0.8/'})
    
    # 설치한 mallet 경로
    mallet_path = 'C:/pythonPrj/jejuModel/mallet-2.0.8/bin/mallet'
    
    model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
    coherencemodel = CoherenceModel(model=model, texts=data_word, dictionary=dictionary, coherence='c_v')
    
    return model, coherencemodel.get_coherence()

### 100개 관광지에 대한 토픽 모델링 진행
    - 추천 대상 100개의 관광지 선정 (2021년 이후 블로그 글 수 순위, 계절별 특성비율, 타 사이트기준 제주도 관광지 순위 등 고려)

In [7]:
tourlist = ['아르떼뮤지엄',
 '산방산',
 '스누피가든',
 '함덕해수욕장',
 '사계해변',
 '섭지코지',
 '도두동 무지개 해안도로',
 '협재해수욕장',
 '동백포레스트',
 '9.81 파크',
 '원동',
 '엉덩물계곡',
 '새별오름',
 '구좌해안로',
 '도치돌 알파카목장',
 '한라산 아래 첫 마을',
 '곶자왈',
 '쇠소깍',
 '송악산',
 '윈드1947 카트 테마파크',
 '뽀로로&타요 테마파크 제주',
 '탑동',
 '이호테우해수욕장',
 '판포포구',
 '용머리해안',
 '백록담',
 '신화테마파크',
 '도두봉',
 '삼양해수욕장',
 '서귀다원',
 '카멜리아힐',
 '안돌오름',
 '광치기해변',
 '비자림',
 '사려니숲길',
 '사진놀이터',
 '관음사',
 '비양도',
 '김녕 청굴물',
 '신창풍차해안도로',
 '가파도',
 '수월이못',
 '해맞이해안로',
 '제주센트럴파크',
 '서귀포해안',
 '중문색달해수욕장',
 '금악오름(왕매)',
 '쁘램 요가',
 '함덕카페거리',
 '브릭캠퍼스',
 '곽지해수욕장',
 '용두암',
 '제주 카카오캠핑',
 '닭머르해안길',
 '금능해수욕장',
 '노을해안로',
 '아침미소목장',
 '서우봉',
 '휴림',
 '도두해안도로',
 '제주토종흑염소목장',
 '취다선리조트',
 '휴애리 자연생활공원',
 '액티브파크',
 '천지연폭포',
 '화조원',
 '민속해안로',
 '김택화미술관',
 '석부작박물관',
 '오조포구',
 '바이나흐튼 크리스마스 박물관',
 '이호테우말등대',
 '평대해변',
 '법환포구',
 '이상한 나라의 앨리스',
 '제주카약',
 '제주도립김창열미술관',
 '혼인지',
 '정방폭포',
 '한라산영실코스',
 '포토갤러리 자연사랑미술관',
 '오설록티뮤지엄',
 '막숙',
 '소천지',
 '신양섭지해수욕장',
 '목장카페 드르쿰다',
 '이로이로공방',
 '수월봉',
 '신화워터파크',
 '제주제트',
 '애월한담해안산책로',
 '김녕해수욕장',
 '수산저수지',
 '주상절리대(중문대포해안)',
 '톨칸이',
 '에코랜드 테마파크',
 '성산일출봉(UNESCO 세계자연유산)',
 '천아숲길 천아계곡',
 '종달리수국길' ,
 '녹산로유채꽃길'] 

In [8]:
len(tourlist)

100

In [18]:
tourDf = df[df['source'].isin(tourlist)]

In [10]:
tourDf = df[df['source'] == '공천포']

In [11]:
tourDf

,source,content
345260,공천포,여행 이튿날 정방폭포 시작 제주 남쪽 가볼만 훑어 성산 가기 했다 제주 남쪽 다양한...
345261,공천포,안녕하세요 입니다 여행가 챙기고 싶은 가지 있어요 바로 일몰 일출 일몰 그래도 시간...
345262,공천포,동백 구경 실컷 하고 버스 타고 올레시장 가기 버스 걸렸당 서귀포 한라산 보인다 쓰...
345263,공천포,제주 여행 광치기해변 일출 소노캄 제주 코스모스 무릎 어떡해 있을까 하면서 일어났는...
345264,공천포,숙소 조식 먹고 숙소 정원 산책 하고 늦잠 자다가 점심시간 가까웠을 나왔다 멀리 움...
...,...,...
345602,공천포,옥돔 갈치 재기 자리 구이 자리 물회 바다 내어주는 다양한 해산물 만든 음식 돼지 ...
345603,공천포,올해 제주 살이 넘어왔다가 제주 매력 빠져 벗어나지 어느새 되었다 이전 제주 여행 ...
345604,공천포,바보 누군가 놀릴 쓰는 아니다 어떠한 대가 없이도 누군가 해자 신의 모든 드러내며 ...
345605,공천포,평소 자주 없는 신체 부위 좋습니다 가령 발목 예쁘네 그런 그런 칭찬 그냥 예쁘다는...


In [21]:
len(tourDf['source'].unique())

100

In [16]:
# seires => DataFrame으로 변환 (원본 데이터)
# tour_OriDf = pd.DataFrame(tourDf)

# 토큰화 진행
tour_tokenized = tokenizedOkt(tourDf['content'])

data_word = tour_tokenized   # 토큰화 데이터
Data_list = tourDf             # 원본 데이터
start=10; limit=11; step=1;  # 모델에서 평가해볼 토픽의 개수 시작, 끝, 증분

  0%|          | 0/347 [00:00<?, ?it/s]

In [19]:
start=20; limit=21; step=1;

In [35]:
model= LDA(data_word, Data_list, 20)

In [27]:
model, topics = LDA(data_word, Data_list, start, limit, step)

Create LDA instance
start: 20, limit: 21, step: 1


  0%|          | 0/1 [00:00<?, ?it/s]

Num Topics = 20  has Coherence Value of 0.42896940074059886


In [29]:
# 결과확인 (공천포)
len(topics)

10

In [38]:
model.show_topics(formatted=False, num_topics=20)

[(0,
  [('카페', 0.47514204545454547),
   ('커피', 0.08948863636363637),
   ('자리', 0.06178977272727273),
   ('지니', 0.05397727272727273),
   ('사장', 0.052556818181818184),
   ('음료', 0.027698863636363636),
   ('종류', 0.027698863636363636),
   ('아메리카노', 0.02556818181818182),
   ('운영', 0.024147727272727272),
   ('실내', 0.022727272727272728)]),
 (1,
  [('도착', 0.07604832977967306),
   ('시간', 0.06965174129353234),
   ('여행', 0.06396588486140725),
   ('거리', 0.057569296375266525),
   ('버스', 0.053304904051172705),
   ('이번', 0.04548685145700071),
   ('타고', 0.0433546552949538),
   ('준비', 0.03980099502487562),
   ('보고', 0.03909026297085998),
   ('이동', 0.03553660270078181)]),
 (2,
  [('메뉴', 0.1191919191919192),
   ('주문', 0.11717171717171718),
   ('방문', 0.10168350168350168),
   ('테이블', 0.07474747474747474),
   ('메뉴판', 0.04915824915824916),
   ('주차', 0.044444444444444446),
   ('내부', 0.0430976430976431),
   ('손님', 0.041750841750841754),
   ('자리', 0.04107744107744108),
   ('바로', 0.035016835016835016)]),
 (3,
  